# PRICE PREDICTABILITY AT ULTRA-HIGH FREQUENCY

### Reproduction

Pour importer l'historique des transactions pour la ou les paires de cryptomonnaies souhaitées, il faut déclarer la liste de paires ainsi que les symboles associés aux intervales de rendements qui seront utilisés tout au long de l'étude. 

Les symboles doivent prendre la forme d'un dictionnaire python avec les symboles en clé et une liste de deux tuples en valeur. Le premier tuple correspondra à l'intervale associé au symbole en clé et le deuxième précisera si les bornes de cet intervales sont incluses ou non.

In [1]:
import numpy as np


asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }

Une fois ces deux variables initialisées on peut importer les données pour un mois ou une journée donnée avec la classe `HistoricalDataCollectorParquet`.

In [2]:
from data.DataCollectors import HistoricalDataCollectorParquet
year = 2025
month = 1
day = 1
historical_collector = HistoricalDataCollectorParquet(pairs=asset_pairs, year=year, month=month, day=day)
historical_collector.collect()

[SYSTEM] Processing BTCUSDT...
[SYSTEM] Parquet data already available for BTCUSDT → data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet → Skipping download.
[SYSTEM] Processing ETHUSDT...
[SYSTEM] Parquet data already available for ETHUSDT → data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet → Skipping download.
[SYSTEM] Processing SOLUSDT...
[SYSTEM] Parquet data already available for SOLUSDT → data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet → Skipping download.


Après avoir collecté les données brutes, on peut les prétraiter (sélection des colonnes pertinentes, calcul des log-prix, des log-rendements) en créant une instance de la classe `DataManager()` avec un niveau d'agrégation donné et utiliser la méthode `block_constructor()` pour construire les blocs de symboles pour une taille donnée.

In [3]:
from data.DataManager import DataManager
data_manager = DataManager(asset_pairs, symbols, year=year, month=month, day=day, aggregation_level=50)
blocks = data_manager.block_constructor(block_size=5, overlapping=False)
blocks['BTCUSDT']

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet


,0,1,2,3,4
0,1,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,1
3,1,1,1,1,1
4,1,1,1,1,1
...,...,...,...,...,...
5993,1,0,0,0,0
5994,0,0,0,0,0
5995,0,0,0,0,0
5996,0,1,1,0,0


Pour mieux visualiser la distribution des blocs pour une paire donnée on peut utiliser la méthode `compute_blocks_frequencies` de la classe `RandomnessAnalysis` initialisée avec un dataframe de blocs.

In [4]:
from main.RandomnessAnalysis import RandomnessAnalysis
s = 2
analyser = RandomnessAnalysis(blocks_df=blocks['BTCUSDT'], s=s)
frequencies_df = analyser.compute_blocks_frequencies()
frequencies_df.set_index('code', inplace=True)
frequencies_df

,block,absolute frequency,relative frequency
code,,,
0,"[0, 0, 0, 0, 0]",1920,0.320107
1,"[0, 0, 0, 0, 1]",239,0.039847
2,"[0, 0, 0, 1, 0]",42,0.007002
3,"[0, 0, 0, 1, 1]",211,0.035178
4,"[0, 0, 1, 0, 0]",37,0.006169
5,"[0, 0, 1, 0, 1]",6,0.001000
6,"[0, 0, 1, 1, 0]",52,0.008670
7,"[0, 0, 1, 1, 1]",225,0.037513
8,"[0, 1, 0, 0, 0]",36,0.006002


On peut également visualiser les histogrammes des fréquences relatives et absolues des blocs avec la fonction `plot_block_frequencies` à partir du dataframe de fréquences produit précedemment.

In [5]:
from utils.VisualizationTools import plot_block_frequencies
plot_block_frequencies(frequencies_df)

La classe `RandomnessAnalysis` a également deux autres méthodes pour effectuer les tests d'hypothèses à partir du dataframe de blocs donné lors de l'initialisation :
- `entropy_bias_test` pour faire un test d'hypothèse de prévisibilité basé sur le biais d'entropie ;
- `KL_divergence_test` pour faire un test d'hypothèse de prévisibilité basé sur la statistique NP.

In [6]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,16170.143873
Quantile 90%,41.421736
Quantile 95%,44.985343
Quantile 99%,52.191395
P-value,0.0
Mean,31
Hypothesis 1,True


Evidemment pour faire le test d'hypothèse de prévibilité utilisant la statistique NP il faut reconstruire des blocs chevauchant en spécifiant `overlapping=True`.

In [7]:
blocks = data_manager.block_constructor(block_size=5, overlapping=True)
analyser = RandomnessAnalysis(blocks_df=blocks['BTCUSDT'], s=s)
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,20228.967701
Quantile 90%,22.30713
Quantile 95%,24.99579
Quantile 99%,30.577914
P-value,0.0
Mean,15
Hypothesis 1,True


Pour effectuer le test de prévisibilité pour différents couples de paramètres (taille de blocs et niveau d'agrégation des transactions), la classe `MultiTester` le permet avec les méthodes `test_by_block_size` et `test_by_aggregation_level`.

In [8]:
from utils.MultiTester import MultiTester
multi_tester = MultiTester(asset='BTCUSDT',symbols=symbols,overlapping=False)
df_test_block = multi_tester.test_by_block_size(test='Entropy Bias',
                                    max_block_size=15,
                                    year=year,
                                    month=month,
                                    day=day,
                                    aggregation_level=50)
df_test_block

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01

,Test statistic,Quantile 99,Quantile 95,Quantile 90,Mean
Block size,,,,,
1,1.729906e+10,6.634897,3.841459,2.705543,1
2,3.849198e+09,11.344867,7.814728,6.251389,3
3,1.559215e+09,18.475307,14.067140,12.017037,7
4,8.045878e+08,30.577914,24.995790,22.307130,15
5,4.735870e+08,52.191395,44.985343,41.421736,31
6,3.019881e+08,92.010024,82.528727,77.745385,63
7,2.037551e+08,166.987390,154.301516,147.804813,127
8,1.425975e+08,310.457388,293.247835,284.335908,255
9,1.025796e+08,588.297794,564.696133,552.373933,511


In [9]:
from utils.VisualizationTools import plot_test
plot_test(x_values=df_test_block.index, 
          y1_values=df_test_block['Test statistic'].values,
          y2_values=df_test_block['Quantile 99'].values,
          test='Entropy Bias',
          x_label='Block size',
          pair='BTCUSDT')

In [10]:
multi_tester = MultiTester(asset='ETHUSDT',symbols=symbols,overlapping=True)
df_test_agg = multi_tester.test_by_aggregation_level(test='NP Statistic',
                                           max_aggregation_level=50,
                                           year=year,
                                           month=month,
                                           day=day,
                                           block_size=5)
df_test_agg

[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01

,Test statistic,Quantile 90,Quantile 95,Quantile 99,Mean
Aggregation level,,,,,
1,227705.225192,22.30713,24.99579,30.577914,15
2,213880.337245,22.30713,24.99579,30.577914,15
3,181430.101693,22.30713,24.99579,30.577914,15
4,162172.016753,22.30713,24.99579,30.577914,15
5,149450.254315,22.30713,24.99579,30.577914,15
6,138556.479424,22.30713,24.99579,30.577914,15
7,122893.046190,22.30713,24.99579,30.577914,15
8,109304.008187,22.30713,24.99579,30.577914,15
9,97025.048207,22.30713,24.99579,30.577914,15


In [11]:
plot_test(x_values=df_test_agg.index, 
          y1_values=df_test_agg['Test statistic'].values,
          y2_values=df_test_agg['Quantile 99'].values,
          test='NP Statistic',
          x_label='Aggregation level',
          pair='ETHUSDT')

Il est également possible de faire répéter le test par taille de blocs ET par niveau d'agrégation afin de voir l'évolution de la valeur des statistiques de test en fonction du couple de paramètres dans un graphique tridimensionnel grâce à la méthode `plot_3D_test_result`.

In [13]:
multi_tester = MultiTester(asset='SOLUSDT',symbols=symbols,overlapping=False)
fig_3D = multi_tester.plot_3D_test_result(asset='SOLUSDT',
                                     test='Entropy Bias',
                                     max_block_size=15,
                                     year=year,month=month,day=day,
                                     max_aggregation_level=50)

[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet


[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet


[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet


[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet


[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet


...

[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet


[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet


[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet


[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet


[SOLUSDT] Grid test 3D: 100%|██████████| 750/750 [02:25<00:00,  5.15it/s]


In [14]:
fig_3D

La fonction `get_assets_properties` permet, une fois les données collectées et traitées, de résumer les caractéristiques des transactions de chaque paire sur le mois ou la journée donné(e).

In [17]:
from utils.Analysis import get_assets_properties
df_prop = get_assets_properties(asset_pairs, s=s, year=year, month=month, day=day, aggregation_level=50)
df_prop

,Mean price,Price standard deviation,Mean return,Return standard deviation,Volume,Mean volume,Standard deviation of volume,Number of transactions
BTCUSDT,94031.677718,479.371948,3.599849e-07,0.000037,1.021105e+04,0.340459,0.791396,29992.0
ETHUSDT,3345.965622,11.119609,2.895938e-07,0.000072,1.882527e+05,7.994087,16.163581,23549.0
SOLUSDT,191.325228,1.657807,1.504519e-06,0.000148,1.643086e+06,95.494969,139.287057,17206.0


La fonction `localization_predictable_intervals` permet également de faire le test d'hypothèses sur une fenêtre temporelle roulante sur un mois ou une journée donné(s) et une paire de cryptomonnaie afin de repérer les intervales de temps où la série de transactions est la plus / moins prévisible.

In [19]:
from utils.Analysis import localization_predictable_intervals
df_window = localization_predictable_intervals(data_manager, "BTCUSDT", test='NP Statistic')
df_window.head(5)

,Timestamp Start,Timestamp End,Test Stat,Quantile 99%,P-value,Hypothesis
Rank,,,,,,
1,2025-01-01 21:57:40.157857,2025-01-01 21:59:12.282466,31.841278,122.837026,0.999634,False
2,2025-01-01 08:50:55.135970,2025-01-01 08:51:49.830841,29.720163,122.837026,0.999886,False
3,2025-01-01 14:37:59.603882,2025-01-01 14:39:44.785281,29.06869,122.837026,0.999923,False
3,2025-01-01 22:39:49.385209,2025-01-01 22:43:09.221646,29.06869,122.837026,0.999923,False
4,2025-01-01 15:45:34.794703,2025-01-01 15:46:29.514209,28.71976,122.837026,0.999938,False


In [20]:
df_window.tail(5)

,Timestamp Start,Timestamp End,Test Stat,Quantile 99%,P-value,Hypothesis
Rank,,,,,,
421,2025-01-01 01:09:25.498225,2025-01-01 01:09:44.815541,0.0,122.837026,1.0,False
421,2025-01-01 12:54:00.557378,2025-01-01 12:55:32.915709,0.0,122.837026,1.0,False
421,2025-01-01 01:05:54.960868,2025-01-01 01:06:19.103987,0.0,122.837026,1.0,False
421,2025-01-01 14:36:12.387692,2025-01-01 14:36:24.895304,0.0,122.837026,1.0,False
421,2025-01-01 23:59:47.619544,2025-01-01 23:59:59.658370,0.0,122.837026,1.0,False


La fonction `intervals_analysis` permet, pour une liste de cryptomonnaies, de présenter la fraction de jours ou d'heures prévisibles selon le test choisi en fonction du niveau d'agrégation des transactions.

In [21]:
from utils.Analysis import intervals_analysis
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
intervals_analysis(pairs=asset_pairs,
                   symbols=symbols,
                   max_aggregation_level=50,
                   year=year,
                   month=month,
                   day=[1,2,3,4,5,6,7,8,9,10],
                   test = 'NP Statistic')

[SYSTEM] Processing 2025-1-1 with aggregation level 1...
[SYSTEM] Processing BTCUSDT...
[SYSTEM] Parquet data already available for BTCUSDT → data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet → Skipping download.
[SYSTEM] Processing ETHUSDT...
[SYSTEM] Parquet data already available for ETHUSDT → data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet → Skipping download.
[SYSTEM] Processing SOLUSDT...
[SYSTEM] Parquet data already available for SOLUSDT → data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet → Skipping download.
[SYSTEM] Processing BNBUSDT...
[SYSTEM] Downloading BNBUSDT-trades-2025-01-01.zip from https://data.binance.vision/data/spot/daily/trades/BNBUSDT/BNBUSDT-trades-2025-01-01.zip ...
[SYSTEM] File downloaded: BNBUSDT-trades-2025-01-01.zip
[SYSTEM] Extracting data/raw_data\BNBUSDT-trades-2025-01-01.zip ...
[SYSTEM] Extraction finished.
[SYSTEM] Converting BNBUSDT-trades-2025-01-01.csv → Parquet ...
[SYSTEM] Saved parquet to: data/raw_data_parquet\BNBUSDT-tr

[SOLUSDT] Grid test 3D:   0%|          | 0/750 [52:23<?, ?it/s]


[SYSTEM] Processing AXSUSDT...
[SYSTEM] Processing RENDERUSDT...
[SYSTEM] Processing 2025-1-1 with aggregation level 3...
[SYSTEM] Processing BTCUSDT...
[SYSTEM] Parquet data already available for BTCUSDT → data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet → Skipping download.
[SYSTEM] Processing ETHUSDT...
[SYSTEM] Parquet data already available for ETHUSDT → data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet → Skipping download.
[SYSTEM] Processing SOLUSDT...
[SYSTEM] Parquet data already available for SOLUSDT → data/raw_data_parquet\SOLUSDT-trades-2025-01-01.parquet → Skipping download.
[SYSTEM] Processing BNBUSDT...
[SYSTEM] Parquet data already available for BNBUSDT → data/raw_data_parquet\BNBUSDT-trades-2025-01-01.parquet → Skipping download.
[SYSTEM] Processing AVAXUSDT...
[SYSTEM] Parquet data already available for AVAXUSDT → data/raw_data_parquet\AVAXUSDT-trades-2025-01-01.parquet → Skipping download.
[SYSTEM] Processing UNIUSDT...
[SYSTEM] Parquet data already avai

Pour étudier les résultats des tests d'hypothèses sur des données synthétiques générées par les modèles λ, OD et TS, on peut utiliser la classe `DataGenerator` qui possède une méthode dédiée pour chacun des modèles :
- `lambda_model`
- `OD_model`
- `TS_model`

Dans chaque méthode, avec l'argument `plots`, on peut préciser si l'on souhaite les deux graphiques de fractions de jours prévisibles, par niveau d'agrégation et par time lag `plots=(True,True)`.

In [22]:
from data.DataGenerator import DataGenerator
data_gen = DataGenerator()
data_gen.lambda_model(test='NP Statistic',
                      overlapping=True,
                      max_aggregation_level=50,
                      max_time_lag=10,
                      n_days=30,
                      plots=(True,True))

Day 1/30
Day 2/30
Day 3/30
Day 4/30
Day 5/30
Day 6/30
Day 7/30
Day 8/30
Day 9/30
Day 10/30
Day 11/30
Day 12/30
Day 13/30
Day 14/30
Day 15/30
Day 16/30
Day 17/30
Day 18/30
Day 19/30
Day 20/30
Day 21/30
Day 22/30
Day 23/30
Day 24/30
Day 25/30
Day 26/30
Day 27/30
Day 28/30
Day 29/30
Day 30/30


In [23]:
data_gen.OD_model(test='NP Statistic',
                  overlapping=True,
                  max_aggregation_level=50,
                  max_time_lag=10,
                  n_days=30,
                  plots=(True,True))

Day 1/30
Day 2/30
Day 3/30
Day 4/30
Day 5/30
Day 6/30
Day 7/30
Day 8/30
Day 9/30
Day 10/30
Day 11/30
Day 12/30
Day 13/30
Day 14/30
Day 15/30
Day 16/30
Day 17/30
128 10
[1.00000000e+03 1.00000000e+03 1.00000000e+03 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.06800000e+02 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.06800000e+02 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.06800000e+02 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.06800000e+02 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.06800000e+02 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.06800000e+02 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.06800000e+02 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.06800000e+02 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.06800000e+02 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.06800000e+02 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.06800000e+02 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.06800000e+02 3.06800000e+02
 3.06800000e+02 3.06800000e+02 3.068000

c:\Users\rabhi\Documents\Master 272 IEF - Dauphine\M2\S2\Gestion Quantitative 2\Price predictability at ultra-high frequency\data\DataGenerator.py:155: RuntimeWarning:

overflow encountered in exp

c:\Users\rabhi\Documents\Master 272 IEF - Dauphine\M2\S2\Gestion Quantitative 2\Price predictability at ultra-high frequency\data\DataGenerator.py:150: RuntimeWarning:

invalid value encountered in subtract

c:\Users\rabhi\Documents\Master 272 IEF - Dauphine\M2\S2\Gestion Quantitative 2\Price predictability at ultra-high frequency\data\DataGenerator.py:154: RuntimeWarning:

invalid value encountered in scalar divide

c:\Users\rabhi\Documents\Master 272 IEF - Dauphine\M2\S2\Gestion Quantitative 2\Price predictability at ultra-high frequency\data\DataGenerator.py:189: RuntimeWarning:

invalid value encountered in scalar divide



Day 18/30
Day 19/30
Day 20/30
Day 21/30
Day 22/30
Day 23/30
Day 24/30
Day 25/30
Day 26/30
Day 27/30
Day 28/30
Day 29/30
Day 30/30


In [24]:
data_gen.TS_model(test='NP Statistic',
                  overlapping=True,
                  max_aggregation_level=50,
                  max_time_lag=10,
                  n_days=30,
                  plots=(True,True))

Day 1/30
Day 2/30
Day 3/30
Day 4/30
Day 5/30
Day 6/30
Day 7/30
Day 8/30
Day 9/30
Day 10/30
Day 11/30
Day 12/30
Day 13/30
Day 14/30
Day 15/30
Day 16/30
Day 17/30
Day 18/30
Day 19/30
Day 20/30
Day 21/30
Day 22/30
Day 23/30
Day 24/30
Day 25/30
Day 26/30
Day 27/30
Day 28/30
Day 29/30
Day 30/30


Pour observer la fraction de jours prévisibles par niveau d'agrégation ou par time lag pour les données générées par les 3 modèles sur un même graphiques, on peut utiliser la méthode `plot_all` et spécifier le graphique souhaité en argument.

In [25]:
data_gen.plot_all(what='aggregation level')

### Extension

La méthode `summarize_transaction_characteristics` de la classe `DataManager` permet d'analyser, pour des paires de cryptomonnaies et une période données, le nombre total de transactions par niveau d'agrégation.

In [29]:
aggregation_levels = [1, 5, 10, 25, 50, 75, 100, 200, 500, 1000]
tx_df = DataManager.summarize_transaction_characteristics(
    pairs=asset_pairs,
    symbols=symbols,
    year=year,
    month=month,
    day=day,
    aggregation_levels=aggregation_levels,
    max_interval_sec=1.0
)
tx_df.loc['BTCUSDT',:]

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01

,total_transactions,avg_interval_sec,fraction_within_1s
aggregation_level,,,
1,439465,0.196601,0.936366
5,241037,0.358450,0.877359
10,139583,0.618984,0.824906
25,59230,1.458694,0.724865
50,29992,2.880682,0.588877
75,20071,4.304660,0.476881
100,15080,5.729436,0.386498
200,7558,11.431951,0.165410
500,3031,28.506457,0.026073


Pour mieux visualiser la relation entre niveau d'agrégation des données et le nombre de transactions, on peut utiliser la méthode `plot_transaction_counts_express`.

In [30]:
fig = DataManager.plot_transaction_counts_express(
    pairs=asset_pairs,
    symbols=symbols,
    year=year, month=month, day=day,
    aggregation_levels=aggregation_levels
)
fig.show()

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01

Le test de prévisibilité croisée a pour but de vérifier si l'observation des blocs d'une paire permet de prédire le futur symbole d'une seconde paire. Pour effectuer le test de prévisibilité croisée, il faut donc ajuster les données des deux paires de cryptomonnaies de manière à ce qu'elles aient un nombre de transactions équivalent. Pour cela, on peut utiliser la méthode `matching_aggregation_for` pour obtenir le niveau d'agrégation optimal de la paire "explicative" à partir du nombre de blocs de la paire dont on essaie de prédire le prochain symbole.

In [31]:
agg_eth = 5
dm_eth = DataManager(['ETHUSDT'], symbols, year, month, day, aggregation_level=agg_eth)
n_blocks_eth = len(dm_eth.datasets['ETHUSDT']['symbol'])
agg_btc = dm_eth.matching_aggregation_for("BTCUSDT", blocks=n_blocks_eth, year=year, month=month, day=day)
dm_btc = DataManager(['BTCUSDT'], symbols, year, month, day, aggregation_level=agg_btc)

[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet


Pour effectuer le test d'hypothèses de prévisibilité croisée, on dispose de la classe `CrossRandomnessAnalysis`. Elle permet en premier lieu de calculer la fréquence des blocs et des symboles construits à partir des deux séries de symboles.

In [33]:
from main.CrossRandomnessAnalysis import CrossRandomnessAnalysis
k = 3   
symbols_btc = dm_btc.datasets['BTCUSDT']['symbol']
symbols_eth = dm_eth.datasets['ETHUSDT']['symbol']
cra = CrossRandomnessAnalysis(
    symbols_context=symbols_btc,
    symbols_target =symbols_eth,
    k=k,
    s=2,
    asset_context='BTCUSDT',
    alpha_context=agg_btc,
    asset_target ='ETHUSDT',
    alpha_target =agg_eth
)
freq_df = cra.compute_cross_frequencies()
freq_df.set_index('block', inplace=True)
freq_df

,code,absolute frequency,relative frequency,asset,alpha,block_size
block,,,,,,
"[0, 0, 0]",0,172113,0.394588,BTCUSDT,2,3
"[0, 0, 1]",1,10615,0.024336,BTCUSDT,2,3
"[0, 1, 0]",2,23111,0.052985,BTCUSDT,2,3
"[0, 1, 1]",3,10148,0.023265,BTCUSDT,2,3
"[1, 0, 0]",4,10615,0.024336,BTCUSDT,2,3
"[1, 0, 1]",5,22645,0.051916,BTCUSDT,2,3
"[1, 1, 0]",6,10148,0.023265,BTCUSDT,2,3
"[1, 1, 1]",7,176789,0.405308,BTCUSDT,2,3
[0],0,77297,0.419015,ETHUSDT,5,1


La classe permet également, avec la méthode `KL_divergence_test`, d'effectuer le test de prévisibilité croisée entre les deux paires à la partir de la statistique NP.

In [35]:
res_kl = cra.KL_divergence_test()   
res_kl

,NP Statistic test
NP Statistic,70.512659
Quantile 90%,12.017037
Quantile 95%,14.06714
Quantile 99%,18.475307
P-value,0.0
Mean,7
Hypothesis 1,True


Tout comme pour les tests univariés, la classe `CrossMultiTester` permet de répéter le test automatiquement pour différentes tailles de blocs et différents niveaux d'agrégation des transactions.

In [36]:
from utils.MultiTester import CrossMultiTester
cmt = CrossMultiTester('BTCUSDT', 'ETHUSDT')

df_blocks = cmt.test_by_block_size(
    test='NP Statistic',
    max_block_size=10,
    alpha_context=5,
    year=year, month=month, day=day
)
df_blocks

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01

,Alpha target,Test statistic,Quantile 90,Quantile 95,Quantile 99,Mean
Block size,,,,,,
1,1,12.834006,2.705543,3.841459,6.634897,1
2,1,16.314787,6.251389,7.814728,11.344867,3
3,1,19.882625,12.017037,14.067140,18.475307,7
4,1,34.974263,22.307130,24.995790,30.577914,15
5,1,51.017687,41.421736,44.985343,52.191395,31
6,1,84.571142,77.745385,82.528727,92.010024,63
7,1,149.378105,147.804813,154.301516,166.987390,127
8,1,298.030062,284.335908,293.247835,310.457388,255
9,1,559.545242,552.373933,564.696133,588.297794,511


In [37]:
df_agg = cmt.test_by_aggregation_level(
    test='NP Statistic',
    aggregation_levels=[1,2,5,10,20,50],
    k=k,
    year=year, month=month, day=day
)
df_agg

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01

,Alpha target,Test statistic,Quantile 90,Quantile 95,Quantile 99,Mean
Aggregation level,,,,,,
1,1,18.218441,12.017037,14.06714,18.475307,7
2,1,14.641598,12.017037,14.06714,18.475307,7
5,1,19.882625,12.017037,14.06714,18.475307,7
10,2,16.916467,12.017037,14.06714,18.475307,7
20,3,18.588476,12.017037,14.06714,18.475307,7
50,9,5.238793,12.017037,14.06714,18.475307,7


La classe permet également de faire le test simultanément pour différents couples de paramètres et d'afficher la statistique de test dans un graphique tridimensionnel.

In [39]:
from utils.VisualizationTools import plot_3D
result_3D = cmt.test_grid(
    test='NP Statistic',
    list_aggregations=np.arange(1,51),
    list_block_sizes=np.arange(1,11),
    year=year, month=month, day=day
)

[BTCUSDT→ETHUSDT] Cross test grid:   0%|          | 0/500 [00:00<?, ?it/s]

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet


[BTCUSDT→ETHUSDT] Cross test grid:   2%|▏         | 10/500 [00:02<01:35,  5.15it/s]

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet


[BTCUSDT→ETHUSDT] Cross test grid:   4%|▍         | 20/500 [00:04<01:32,  5.20it/s]

...

[BTCUSDT→ETHUSDT] Cross test grid:  96%|█████████▌| 480/500 [01:16<00:02,  8.90it/s]

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet


[BTCUSDT→ETHUSDT] Cross test grid:  98%|█████████▊| 489/500 [01:18<00:01,  9.07it/s]

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-01-01.parquet


[BTCUSDT→ETHUSDT] Cross test grid: 100%|██████████| 500/500 [01:19<00:00,  6.29it/s]


In [40]:
plot_3D(result_3D, test='NP Statistic')